In [1]:
using Serialization: serialize, deserialize
using MPI
MPI.Init()

MPI.ThreadLevel(2)

In [7]:
comm = MPI.COMM_WORLD
println("Hello world, I am $(MPI.Comm_rank(comm) + 1) of $(MPI.Comm_size(comm))")
MPI.Barrier(comm)

Hello world, I am 1 of 1


In [8]:
# ccall((:braid_Hello, "./libbraid.so"), Cint, ());
@ccall "./libbraid.so".braid_Hello(comm::Ptr{MPI.MPI_Comm})::Cint

Hello world, from rank 32767!


0

In [9]:
# C_stdout = Libc.FILE(Libc.RawFD(1), "w")  # corresponds to C standard output
file = open("./cstdout.txt", "w")
C_stdout = Libc.FILE(file)
@ccall fprintf(C_stdout::Ptr{Cvoid}, "Hello, stdout!\n"::Cstring)::Cint

15

In [10]:
# Data structures required by XBraid
struct braid_app
    comm::MPI.Comm      # MPI communicator
    bufsize::Integer    # default size of a braid_vector allocation
    vec_refs::IdDict    # dictionary which stores global references to
                        # braid_vectors, to keep them from being garbage
                        # collected while they only exist as C pointers
                        # keys are objectid hashes, and values are references
end

mutable struct braid_vector
    data
end

vec = braid_vector(zeros(4))
app = braid_app(MPI.COMM_WORLD, Base.summarysize(vec), IdDict())

braid_app(MPI.Comm(Ptr{Nothing} @0x00007f43c31efc20), 80, IdDict{Any, Any}())

In [16]:
function my_pack!(_app, u, buffer::Ptr{Cvoid}, status::Ptr{Cvoid})::Cint
    # println("pack")
    vec_ref = unsafe_pointer_to_objref(vec)
    data_arr = unsafe_wrap(Vector{UInt8}, Base.unsafe_convert(Ptr{UInt8}, buffer), app.bufsize)
    buff = IOBuffer(data_arr, write=true, maxsize=app.bufsize)
    serialize(buff, vec_ref)
    return 0
end

function my_unpack!(_app, buffer::Ptr{Cvoid}, u_ptr::Ptr{Ptr{Cvoid}}, status::Ptr{Cvoid})::Cint
    # println("unpack")
    data_arr = unsafe_wrap(Vector{UInt8}, Base.unsafe_convert(Ptr{UInt8}, buffer), app.bufsize)
    buff = IOBuffer(data_arr, read=true, maxsize=app.bufsize)
    # unpack the buffer into a julia struct, then register with IdDict and store in u_ptr
    u = deserialize(buff)
    app.vec_refs[objectid(u)] = u # store reference to u
    unsafe_store!(u_ptr, pointer_from_objref(u))

    # vec = Base.unsafe_convert(Ptr{Cvoid}, Ref(vec_ref))
    return 0
end

function my_buffsize!(_app, size::Ptr{Cint})::Cint
    # println("size")
    unsafe_store!(size, app.bufsize)
    return 0
end

function my_access!(_app, u, status::Ptr{Cvoid})::Cint
    println("access")
    u = unsafe_pointer_to_objref(u)
    println(u)
    return 0
end

function my_init!(_app, t::Real, u_ptr::Ptr{Ptr{Cvoid}})::Cint
    println("init")
    u = braid_vector(zeros(4))
    # Since app is in the global scope, u will survive garbage collection even after it goes out of scope here
    app.vec_refs[objectid(u)] = u # store reference to u
    unsafe_store!(u_ptr, pointer_from_objref(u))
    return 0
end

function my_free!(_app, u::Ptr{Cvoid})::Cint
    println("free")
    u_ref = unsafe_pointer_to_objref(u)
    # removing the global reference to u will allow garbage collection to delete it
    pop!(app.vec_refs, objectid(u_ref))
    u = C_NULL
    return 0
end

function my_sum!(_app, alpha::Real, x::Ptr{Cvoid}, beta::Real, y::Ptr{Cvoid})::Cint
    x_ref = unsafe_pointer_to_objref(x)
    y_ref = unsafe_pointer_to_objref(y)
    # @. vectorizes every operation in this line, meaning this assignment is in-place!
    @. y_ref.data = alpha * x_ref.data + beta * y_ref.data
    return 0
end

function my_clone!(_app, u::Ptr{Cvoid}, v_ptr::Ptr{Ptr{Cvoid}})::Cint
    u_ref = unsafe_pointer_to_objref(u)
    v_ref = deepcopy(u_ref) # this allocates v_ref and copies u_ref
    # need to add reference to ref_ids
    app.vec_refs[objectid(v_ref)] = v_ref
    unsafe_store!(v_ptr, pointer_from_objref(v_ref))
    return 0
end

pack_c = @cfunction(my_pack!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}))
unpack_c = @cfunction(my_unpack!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}))
buffsize_c = @cfunction(my_buffsize!, Cint, (Ptr{Cvoid}, Ptr{Cint}))

access_c = @cfunction(my_access!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}))
init_c = @cfunction(my_init!, Cint, (Ptr{Cvoid}, Cdouble, Ptr{Ptr{Cvoid}}))
free_c = @cfunction(my_free!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}))
sum_c = @cfunction(my_sum!, Cint, (Ptr{Cvoid}, Cdouble, Ptr{Cvoid}, Cdouble, Ptr{Cvoid}))
clone_c = @cfunction(my_clone!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Ptr{Cvoid}}))

Ptr{Nothing} @0x00007f43d8d7ea00

In [17]:
using Base.Libc: FILE
file = open("./test_init_access.txt", "w")
@ccall "./libbraid.so".braid_TestInitAccess(C_NULL::Ptr{Cvoid}, app.comm::Ptr{MPI.MPI_Comm}, C_NULL::Ptr{Cvoid}, 0.::Cdouble,
                                            init_c::Ptr{Cvoid}, access_c::Ptr{Cvoid}, free_c::Ptr{Cvoid})::Cint
close(file)

In [10]:
u = Ref([1, 2, 3, 4])
v = [2, 2, 3, 3]
u_ptr = Base.Libc.malloc(8)
u_ptr = Base.unsafe_convert(Ptr{Ptr{Cvoid}}, u_ptr)

Ptr{Ptr{Nothing}} @0x000000000185c230

In [11]:
my_init!(nothing, 0., u_ptr)

init


0

In [12]:
app.vec_refs

IdDict{Any, Any} with 1 entry:
  0xb5ffab017e576c17 => braid_vector([0.0, 0.0, 0.0, 0.0])

In [13]:
my_free!(nothing, unsafe_load(u_ptr))

free


0

In [14]:
app.vec_refs

IdDict{Any, Any}()

In [8]:
function init(a_ptr::Ptr{Ptr{Cvoid}})
    a = braid_vector([1, 2, 3])
    # add reference to IdDict to maintain global reference,
    # or else garbage collection may destroy this later!
    ref_ids[objectid(a)] = a
    unsafe_store!(a_ptr, pointer_from_objref(a))
    pointer_from_objref(a)
    return 0
end

function free!(a_ptr::Ptr{Ptr{Cvoid}})
    a = unsafe_pointer_to_objref(a_ptr)
    println(ref_ids[objectid(a)])
    pop!(ref_ids, objectid(a))
    a_ptr = C_NULL
end

free! (generic function with 1 method)

In [9]:
a_ptr = init()
b_ptr = init()
show(ref_ids)
GC.gc()
free!(a_ptr)
free!(b_ptr)

IdDict{Any, Any}(0xa2426d04bf525d91 => braid_vector(

[1, 2, 3]), 0xd99a08ae2e018c9b => braid_vector([1, 2, 3]))braid_vector

([1, 2, 3])
braid_vector([1, 2, 3])


Ptr{Nothing} @0x0000000000000000

In [12]:
ref_ids

IdDict{Any, Any}()